In [193]:
import pandas as pd
import sqlalchemy as sqla
import pyodbc
from enum import Enum, auto

from importlib import reload

from country_correction import adjust_country_names
# We use autoreload so that it automatically reloads the changes we made in modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST     = os.getenv('DB_HOST')
DB_PORT     = os.getenv('DB_PORT')
DB_NAME     = os.getenv('DB_NAME')

FOLDER = 'Dataset'
FILE_COVID = FOLDER + '/COVID_2019.csv'
FILE_EBOLA = FOLDER + '/EBOLA_2014_2016.csv'
FILE_H1N1  = FOLDER + '/H1N1_2009.csv'
FILE_SARS  = FOLDER + '/SARS_2003.csv'

In [195]:
# Define column names
COL_DATE        = 'Date'
COL_COUNTRY     = 'Country'
COL_STATE       = 'ProvinceState'
COL_WHOREGION   = 'WHORegion'
COL_LATITUDE    = 'Latitude'
COL_LONGITUDE   = 'Longitude'
COL_CONFIRMED   = 'Confirmed'
COL_DEATHS      = 'Deaths'
COL_RECOVERED   = 'Recovered'
COL_ACTIVE      = 'Active'

In [196]:
class Pandemics(Enum):
    SARS = 1
    H1N1 = 2
    EBOLA = 3
    COVID = 4

In [197]:
connection_url = f"mssql+pymssql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

engine = sqla.create_engine(connection_url)
try:
    # Tests the SQL Alchemy connection
    with engine.connect() as conn:
        result = conn.execute(sqla.text("SELECT TOP 10 * FROM Dim.Date"))
        for row in result:
            print(row)
        pass

except Exception as e:
    print(f"SQLAlchemy connection error: {e}")

(20000101, datetime.date(2000, 1, 1), 1, 'st', 7, 'Saturday', 'SAT', 'S', 1, 1, 1, 1, 1, 'January', 'JAN', 'J', 1, 'First', 2000, '012000', '2000JAN', True, False)
(20000102, datetime.date(2000, 1, 2), 2, 'nd', 1, 'Sunday', 'SUN', 'S', 2, 2, 2, 2, 1, 'January', 'JAN', 'J', 1, 'First', 2000, '012000', '2000JAN', True, False)
(20000103, datetime.date(2000, 1, 3), 3, 'rd', 2, 'Monday', 'MON', 'M', 3, 3, 2, 2, 1, 'January', 'JAN', 'J', 1, 'First', 2000, '012000', '2000JAN', False, False)
(20000104, datetime.date(2000, 1, 4), 4, 'th', 3, 'Tuesday', 'TUE', 'T', 4, 4, 2, 2, 1, 'January', 'JAN', 'J', 1, 'First', 2000, '012000', '2000JAN', False, False)
(20000105, datetime.date(2000, 1, 5), 5, 'th', 4, 'Wednesday', 'WED', 'W', 5, 5, 2, 2, 1, 'January', 'JAN', 'J', 1, 'First', 2000, '012000', '2000JAN', False, False)
(20000106, datetime.date(2000, 1, 6), 6, 'th', 5, 'Thursday', 'THU', 'T', 6, 6, 2, 2, 1, 'January', 'JAN', 'J', 1, 'First', 2000, '012000', '2000JAN', False, False)
(20000107, datet

# Covid

In [198]:
covid = pd.read_csv(FILE_COVID)
covid['Date'] = pd.to_datetime(covid['Date']).dt.strftime('%Y-%m-%d')
covid['PandemicId'] = Pandemics.COVID.value
# covid.drop([
#         'Date',
#         'Confirmed',
#         'Deaths',
#         'Recovered',
#         'Active'
#     ], axis='columns', inplace=True)
covid.rename(columns={
    'Province/State': COL_STATE,
    'Country/Region': COL_COUNTRY,
    'Lat': COL_LATITUDE,
    'Long': COL_LONGITUDE,
    'WHO Region': COL_WHOREGION
}, inplace=True)
adjust_country_names(covid)
covid.drop_duplicates(inplace=True)
covid

# with engine.begin() as conn:
#     covid.to_sql(
#         name='Country',
#         schema='Dim',
#         con=conn,
#         index_label='CountryID',
#         if_exists='replace', 
#     )
dim_country = set(covid['Country'].unique())
dim_country
covid

,ProvinceState,Country,Latitude,Longitude,Date,Confirmed,Deaths,Recovered,Active,WHORegion,PandemicId
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean,4
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,0,Europe,4
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0,Africa,4
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0,Europe,4
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0,Africa,4
...,...,...,...,...,...,...,...,...,...,...,...
48802,NaN,Sao Tome and Principe,0.186400,6.613100,2020-07-26,863,14,696,153,Africa,4
48803,NaN,Yemen,15.552727,48.516388,2020-07-26,1681,479,797,405,Eastern Mediterranean,4
48804,NaN,Comoros,-11.645500,43.333300,2020-07-26,354,7,328,19,Africa,4
48805,NaN,Tajikistan,38.861000,71.276100,2020-07-26,7192,59,5970,1163,Europe,4


# Ebola

In [199]:
ebola = pd.read_csv(FILE_EBOLA)
ebola['Date'] = pd.to_datetime(ebola['Date']).dt.strftime('%Y-%m-%d')
ebola['PandemicId'] = Pandemics.EBOLA.value
ebola.rename(columns={
    'Cumulative no. of confirmed, probable and suspected cases':  COL_CONFIRMED,
    'Cumulative no. of confirmed, probable and suspected deaths': COL_DEATHS,
}, inplace=True)
adjust_country_names(ebola)
ebola
new_country = set(ebola['Country'].unique())
diff = new_country - dim_country
diff

set()

# H1N1

In [200]:
h1n1 = pd.read_csv(FILE_H1N1)
h1n1['Date'] = pd.to_datetime(h1n1['Date']).dt.strftime('%Y-%m-%d')
h1n1['PandemicId'] = Pandemics.H1N1.value
h1n1.rename(columns={
    'Cumulative no. of cases':  COL_CONFIRMED,
    'Cumulative no. of deaths': COL_DEATHS,
}, inplace=True)
adjust_country_names(h1n1)
h1n1
new_country = set(h1n1['Country'].unique())
diff = new_country - dim_country
diff

{'Brunei Darussalam', 'Cook Island', 'Palau', 'Samoa', 'Vanuatu'}

# SARS

In [201]:
sars = pd.read_csv(FILE_SARS)
sars['Date'] = pd.to_datetime(sars['Date']).dt.strftime('%Y-%m-%d')
sars['PandemicId'] = Pandemics.SARS.value
sars.rename(columns={
    'Cumulative number of case(s)': COL_CONFIRMED,
    'Number of deaths':             COL_DEATHS,
    'Number recovered':             COL_RECOVERED,
}, inplace=True)
adjust_country_names(sars)
sars
new_country = set(sars['Country'].unique())
diff = new_country - dim_country
diff

set()

# Merging Data

In [202]:
df = pd.concat([covid,ebola,h1n1,sars], ignore_index=True)
df

,ProvinceState,Country,Latitude,Longitude,Date,Confirmed,Deaths,Recovered,Active,WHORegion,PandemicId
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0.0,0.0,0.0,0.0,Eastern Mediterranean,4
1,NaN,Albania,41.15330,20.168300,2020-01-22,0.0,0.0,0.0,0.0,Europe,4
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0.0,0.0,0.0,0.0,Africa,4
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0.0,0.0,0.0,0.0,Europe,4
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0.0,0.0,0.0,0.0,Africa,4
...,...,...,...,...,...,...,...,...,...,...,...
56315,NaN,Switzerland,NaN,NaN,2003-07-11,1.0,0.0,1.0,NaN,NaN,1
56316,NaN,Thailand,NaN,NaN,2003-07-11,9.0,2.0,7.0,NaN,NaN,1
56317,NaN,United Kingdom,NaN,NaN,2003-07-11,4.0,0.0,4.0,NaN,NaN,1
56318,NaN,United States,NaN,NaN,2003-07-11,75.0,0.0,67.0,NaN,NaN,1


## DimCountry